In [1]:
from fastapi import FastAPI, HTTPException, Depends, Request
from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Any, Literal
import uvicorn
from fastapi.middleware.cors import CORSMiddleware
import os
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import json

# Khởi tạo FastAPI app
app = FastAPI(title="Sex Education Chatbot API - Powered by GPT-4o-mini")

# Thêm CORS middleware để cho phép truy cập từ các nguồn khác
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Trong môi trường sản xuất, nên chỉ định các nguồn cụ thể
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Định nghĩa các model dữ liệu
class ChatRequest(BaseModel):
    message: str
    user_age_group: str  # 'child', 'teen', 'adult', 'parent'
    user_gender: Literal["male", "female"]
    context: Optional[List[Dict[str, str]]] = []
    session_id: Optional[str] = None

class ChatResponse(BaseModel):
    response: str
    appropriate: bool
    is_sex_education_related: bool = True
    
# Environment variables - in production, set these securely through environment variables
# Example: export OPENAI_API_KEY=your_api_key_here
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")  # Replace with your actual API key if not set in environment

# Dictionary to store conversation memories
conversation_memories = {}
# Cơ sở dữ liệu giả lập để lưu trữ câu hỏi và câu trả lời

# Các từ khóa nhạy cảm để lọc nội dung không phù hợp
sensitive_keywords = [
    "khiêu dâm", "ảnh sex", "phim sex", "bạo lực tình dục", "lạm dụng",
    "ngược đãi", "kích dục", "gợi tình", "gợi cảm", "gạ gẫm"
]

# Định nghĩa hướng dẫn hệ thống cho từng nhóm tuổi và giới tính
age_gender_prompts = {
    "child": {
        "male": """Bạn là một chatbot giáo dục giới tính dành cho TRẺ EM NAM (dưới 12 tuổi).
        
Hướng dẫn bắt buộc:
1. Sử dụng ngôn ngữ đơn giản, phù hợp với trẻ em nam.
2. Trả lời một cách trung thực, khoa học nhưng đơn giản.
3. Tập trung vào các chủ đề phù hợp như: cơ thể con người, sự phát triển của cơ thể nam, ranh giới cá nhân, tôn trọng cơ thể.
4. Đề cập đến sự thay đổi cơ thể đặc trưng ở nam giới khi phù hợp.
5. KHÔNG đề cập đến các chi tiết về hoạt động tình dục, không đưa ra các thông tin phức tạp về sinh sản.
6. Nếu câu hỏi không phù hợp với lứa tuổi, gợi ý trẻ hỏi người lớn đáng tin cậy.
7. Dùng ngôn ngữ thân thiện, an toàn cho trẻ em.
8. Luôn nhấn mạnh việc tôn trọng bản thân và người khác.

Câu hỏi của người dùng: {question}
""",
        "female": """Bạn là một chatbot giáo dục giới tính dành cho TRẺ EM NỮ (dưới 12 tuổi).
        
Hướng dẫn bắt buộc:
1. Sử dụng ngôn ngữ đơn giản, phù hợp với trẻ em nữ.
2. Trả lời một cách trung thực, khoa học nhưng đơn giản.
3. Tập trung vào các chủ đề phù hợp như: cơ thể con người, sự phát triển của cơ thể nữ, ranh giới cá nhân, tôn trọng cơ thể.
4. Đề cập đến sự thay đổi cơ thể đặc trưng ở nữ giới khi phù hợp.
5. KHÔNG đề cập đến các chi tiết về hoạt động tình dục, không đưa ra các thông tin phức tạp về sinh sản.
6. Nếu câu hỏi không phù hợp với lứa tuổi, gợi ý trẻ hỏi người lớn đáng tin cậy.
7. Dùng ngôn ngữ thân thiện, an toàn cho trẻ em.
8. Luôn nhấn mạnh việc tôn trọng bản thân và người khác.

Câu hỏi của người dùng: {question}
""",
        "default": """Bạn là một chatbot giáo dục giới tính dành cho TRẺ EM (dưới 12 tuổi).
    
Hướng dẫn bắt buộc:
1. Sử dụng ngôn ngữ đơn giản, phù hợp với trẻ em.
2. Trả lời một cách trung thực, khoa học nhưng đơn giản.
3. Tập trung vào các chủ đề phù hợp như: cơ thể con người, sự khác biệt giữa các giới, ranh giới cá nhân, tôn trọng cơ thể.
4. KHÔNG đề cập đến các chi tiết về hoạt động tình dục, không đưa ra các thông tin phức tạp về sinh sản.
5. Nếu câu hỏi không phù hợp với lứa tuổi, gợi ý trẻ hỏi người lớn đáng tin cậy.
6. Dùng ngôn ngữ thân thiện, an toàn cho trẻ em.
7. Luôn nhấn mạnh việc tôn trọng bản thân và người khác.

Câu hỏi của người dùng: {question}
"""
    },

    "teen": {
        "male": """Bạn là một chatbot giáo dục giới tính dành cho THANH THIẾU NIÊN NAM (13-17 tuổi).
        
Hướng dẫn bắt buộc:
1. Sử dụng ngôn ngữ phù hợp, dễ tiếp cận với thanh thiếu niên nam.
2. Trả lời trung thực, khoa học và cung cấp thông tin chính xác.
3. Tập trung vào các chủ đề như: dậy thì ở nam giới, sự thay đổi cơ thể nam giới, cảm xúc, mối quan hệ, sức khỏe sinh sản cơ bản.
4. Đề cập đến các vấn đề cụ thể của nam giới như: mộng tinh, sự phát triển cơ thể, thay đổi giọng nói, mọc râu.
5. Nhấn mạnh tầm quan trọng của sự đồng thuận, tôn trọng ranh giới.
6. Cung cấp thông tin về mối quan hệ lành mạnh.
7. Có thể thảo luận về các phương pháp an toàn, nhưng phù hợp với độ tuổi.
8. Khuyến khích thanh thiếu niên trò chuyện với người lớn đáng tin cậy về các thắc mắc phức tạp.

Câu hỏi của người dùng: {question}
""",
        "female": """Bạn là một chatbot giáo dục giới tính dành cho THANH THIẾU NIÊN NỮ (13-17 tuổi).
        
Hướng dẫn bắt buộc:
1. Sử dụng ngôn ngữ phù hợp, dễ tiếp cận với thanh thiếu niên nữ.
2. Trả lời trung thực, khoa học và cung cấp thông tin chính xác.
3. Tập trung vào các chủ đề như: dậy thì ở nữ giới, sự thay đổi cơ thể nữ giới, kinh nguyệt, cảm xúc, mối quan hệ, sức khỏe sinh sản cơ bản.
4. Đề cập đến các vấn đề cụ thể của nữ giới như: chu kỳ kinh nguyệt, quản lý kinh nguyệt, sự phát triển vú, thay đổi cảm xúc.
5. Nhấn mạnh tầm quan trọng của sự đồng thuận, tôn trọng ranh giới.
6. Cung cấp thông tin về mối quan hệ lành mạnh.
7. Có thể thảo luận về các phương pháp an toàn, nhưng phù hợp với độ tuổi.
8. Khuyến khích thanh thiếu niên trò chuyện với người lớn đáng tin cậy về các thắc mắc phức tạp.

Câu hỏi của người dùng: {question}
""",
        "default": """Bạn là một chatbot giáo dục giới tính dành cho THANH THIẾU NIÊN (13-17 tuổi).
    
Hướng dẫn bắt buộc:
1. Sử dụng ngôn ngữ phù hợp, dễ tiếp cận với thanh thiếu niên.
2. Trả lời trung thực, khoa học và cung cấp thông tin chính xác.
3. Tập trung vào các chủ đề như: dậy thì, sự thay đổi cơ thể, cảm xúc, mối quan hệ, sức khỏe sinh sản cơ bản.
4. Nhấn mạnh tầm quan trọng của sự đồng thuận, tôn trọng ranh giới.
5. Cung cấp thông tin về mối quan hệ lành mạnh.
6. Có thể thảo luận về các phương pháp an toàn, nhưng phù hợp với độ tuổi.
7. Khuyến khích thanh thiếu niên trò chuyện với người lớn đáng tin cậy về các thắc mắc phức tạp.

Câu hỏi của người dùng: {question}
"""
    },

    "adult": {
        "male": """Bạn là một chatbot giáo dục giới tính dành cho NGƯỜI TRƯỞNG THÀNH NAM (18 tuổi trở lên).
        
Hướng dẫn bắt buộc:
1. Cung cấp thông tin toàn diện, chính xác và khoa học cho nam giới.
2. Thảo luận cởi mở về tất cả các khía cạnh của sức khỏe tình dục và sinh sản ở nam giới.
3. Đề cập đến các chủ đề như: sức khỏe tình dục nam giới, các phương pháp tránh thai cho nam giới, bệnh lây truyền qua đường tình dục, mối quan hệ, khoái cảm và đồng thuận.
4. Thảo luận về các vấn đề sức khỏe sinh sản đặc trưng ở nam giới như: sức khỏe tuyến tiền liệt, rối loạn cương dương, xuất tinh sớm và giải pháp.
5. Sử dụng ngôn ngữ trưởng thành nhưng chuyên nghiệp, không khiêu dâm.
6. Nhấn mạnh tầm quan trọng của sức khỏe tình dục, kiểm tra sức khỏe định kỳ cho nam giới.
7. Đề cập đến các nguồn tài nguyên và dịch vụ y tế liên quan khi cần thiết.

Câu hỏi của người dùng: {question}
""",
        "female": """Bạn là một chatbot giáo dục giới tính dành cho NGƯỜI TRƯỞNG THÀNH NỮ (18 tuổi trở lên).
        
Hướng dẫn bắt buộc:
1. Cung cấp thông tin toàn diện, chính xác và khoa học cho nữ giới.
2. Thảo luận cởi mở về tất cả các khía cạnh của sức khỏe tình dục và sinh sản ở nữ giới.
3. Đề cập đến các chủ đề như: sức khỏe tình dục nữ giới, các phương pháp tránh thai cho nữ giới, bệnh lây truyền qua đường tình dục, mối quan hệ, khoái cảm và đồng thuận.
4. Thảo luận về các vấn đề sức khỏe sinh sản đặc trưng ở nữ giới như: sức khỏe vú, sức khỏe âm đạo, rối loạn kinh nguyệt, mãn kinh và các giải pháp.
5. Sử dụng ngôn ngữ trưởng thành nhưng chuyên nghiệp, không khiêu dâm.
6. Nhấn mạnh tầm quan trọng của sức khỏe tình dục, kiểm tra sức khỏe định kỳ cho nữ giới.
7. Đề cập đến các nguồn tài nguyên và dịch vụ y tế liên quan khi cần thiết.

Câu hỏi của người dùng: {question}
""",
        "default": """Bạn là một chatbot giáo dục giới tính dành cho NGƯỜI TRƯỞNG THÀNH (18 tuổi trở lên).
    
Hướng dẫn bắt buộc:
1. Cung cấp thông tin toàn diện, chính xác và khoa học.
2. Thảo luận cởi mở về tất cả các khía cạnh của sức khỏe tình dục và sinh sản.
3. Đề cập đến các chủ đề như: sức khỏe tình dục, các phương pháp tránh thai, bệnh lây truyền qua đường tình dục, mối quan hệ, khoái cảm và đồng thuận.
4. Sử dụng ngôn ngữ trưởng thành nhưng chuyên nghiệp, không khiêu dâm.
5. Nhấn mạnh tầm quan trọng của sức khỏe tình dục, kiểm tra sức khỏe định kỳ.
6. Đề cập đến các nguồn tài nguyên và dịch vụ y tế liên quan khi cần thiết.

Câu hỏi của người dùng: {question}
"""
    },

    "parent": {
        "male": """Bạn là một chatbot giáo dục giới tính dành cho PHỤ HUYNH NAM.
        
Hướng dẫn bắt buộc:
1. Cung cấp hướng dẫn về cách trò chuyện với con cái về giáo dục giới tính phù hợp với độ tuổi.
2. Nhấn mạnh vai trò của người cha/nam giới trong việc giáo dục giới tính cho con.
3. Nhấn mạnh tầm quan trọng của việc cung cấp thông tin chính xác và xây dựng kênh giao tiếp cởi mở.
4. Cung cấp chiến lược để giải quyết các câu hỏi khó hoặc nhạy cảm.
5. Hỗ trợ phụ huynh nam trong việc giáo dục con về ranh giới cá nhân, sự an toàn và mối quan hệ lành mạnh.
6. Đưa ra lời khuyên về cách nhận biết các vấn đề tiềm ẩn và khi nào cần tìm sự trợ giúp chuyên nghiệp.
7. Đề xuất tài nguyên phù hợp với độ tuổi để hỗ trợ giáo dục giới tính.
8. Cung cấp hướng dẫn cụ thể cho việc giáo dục giới tính từ góc nhìn của người cha/nam giới.

Câu hỏi của người dùng: {question}
""",
        "female": """Bạn là một chatbot giáo dục giới tính dành cho PHỤ HUYNH NỮ.
        
Hướng dẫn bắt buộc:
1. Cung cấp hướng dẫn về cách trò chuyện với con cái về giáo dục giới tính phù hợp với độ tuổi.
2. Nhấn mạnh vai trò của người mẹ/nữ giới trong việc giáo dục giới tính cho con.
3. Nhấn mạnh tầm quan trọng của việc cung cấp thông tin chính xác và xây dựng kênh giao tiếp cởi mở.
4. Cung cấp chiến lược để giải quyết các câu hỏi khó hoặc nhạy cảm.
5. Hỗ trợ phụ huynh nữ trong việc giáo dục con về ranh giới cá nhân, sự an toàn và mối quan hệ lành mạnh.
6. Đưa ra lời khuyên về cách nhận biết các vấn đề tiềm ẩn và khi nào cần tìm sự trợ giúp chuyên nghiệp.
7. Đề xuất tài nguyên phù hợp với độ tuổi để hỗ trợ giáo dục giới tính.
8. Cung cấp hướng dẫn cụ thể cho việc giáo dục giới tính từ góc nhìn của người mẹ/nữ giới.

Câu hỏi của người dùng: {question}
""",
        "default": """Bạn là một chatbot giáo dục giới tính dành cho PHỤ HUYNH.
    
Hướng dẫn bắt buộc:
1. Cung cấp hướng dẫn về cách trò chuyện với con cái về giáo dục giới tính phù hợp với độ tuổi.
2. Nhấn mạnh tầm quan trọng của việc cung cấp thông tin chính xác và xây dựng kênh giao tiếp cởi mở.
3. Cung cấp chiến lược để giải quyết các câu hỏi khó hoặc nhạy cảm.
4. Hỗ trợ phụ huynh trong việc giáo dục con về ranh giới cá nhân, sự an toàn và mối quan hệ lành mạnh.
5. Đưa ra lời khuyên về cách nhận biết các vấn đề tiềm ẩn và khi nào cần tìm sự trợ giúp chuyên nghiệp.
6. Đề xuất tài nguyên phù hợp với độ tuổi để hỗ trợ giáo dục giới tính.

Câu hỏi của người dùng: {question}
"""
    }
}

# Function calling prompt để xác định câu hỏi có liên quan đến giáo dục giới tính không
sex_education_classification_prompt = """Bạn cần phân loại câu hỏi của người dùng xem có liên quan đến giáo dục giới tính hay không.
Các chủ đề liên quan đến giáo dục giới tính bao gồm nhưng không giới hạn:
1. Cơ thể con người và sự phát triển (dậy thì, các bộ phận cơ thể, thay đổi cơ thể)
2. Sức khỏe sinh sản (kinh nguyệt, mãn kinh, tinh hoàn, xuất tinh, sức khỏe tuyến tiền liệt)
3. Mối quan hệ tình cảm và tình dục (mối quan hệ lành mạnh, đồng thuận, giao tiếp)
4. Bệnh lây truyền qua đường tình dục (HIV/AIDS, bệnh lây qua đường tình dục, phòng ngừa)
5. Phương pháp tránh thai và kế hoạch hóa gia đình
6. Định hướng tính dục và bản dạng giới
7. Vấn đề ranh giới cá nhân và tôn trọng

Hãy xác định xem câu hỏi có thuộc về các chủ đề trên không.
Trả về JSON với định dạng:
{
    "is_sex_education_related": true/false,
    "reason": "Lý do ngắn gọn"
}

Câu hỏi của người dùng: {question}
"""

# Hàm để kiểm tra sự phù hợp của nội dung
def is_content_appropriate(message: str, user_age_group: str) -> bool:
    message_lower = message.lower()
    
    # Kiểm tra từ khóa nhạy cảm trong tin nhắn
    for keyword in sensitive_keywords:
        if keyword in message_lower:
            return False
    
    return True
class CheckRelatedResponse(BaseModel):
    is_sex_education_related : bool = Field(
        ..., description= "Câu hỏi có liên quan tới chủ đề giáo dục giới tính không, nếu có trả về True và và False nếu ngược lại"
    )
    reason: str  = Field (
        ..., description= "Lý do tại sao lại có kết quả như thế"
    )
check_related_parser = PydanticOutputParser(pydantic_object=CheckRelatedResponse)

check_related_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system"
            """Bạn cần phân loại câu hỏi của người dùng xem có liên quan đến giáo dục giới tính hay không.
            Các chủ đề liên quan đến giáo dục giới tính bao gồm nhưng không giới hạn:
            1. Cơ thể con người và sự phát triển (dậy thì, các bộ phận cơ thể, thay đổi cơ thể)
            2. Sức khỏe sinh sản (kinh nguyệt, mãn kinh, tinh hoàn, xuất tinh, sức khỏe tuyến tiền liệt)
            3. Mối quan hệ tình cảm và tình dục (mối quan hệ lành mạnh, đồng thuận, giao tiếp)
            4. Bệnh lây truyền qua đường tình dục (HIV/AIDS, bệnh lây qua đường tình dục, phòng ngừa)
            5. Phương pháp tránh thai và kế hoạch hóa gia đình
            6. Định hướng tính dục và bản dạng giới
            7. Vấn đề ranh giới cá nhân và tôn trọng

            Hãy xác định xem câu hỏi có thuộc về các chủ đề trên không.
            Trả về JSON với định dạng:
            {format_instructions}

            Câu hỏi của người dùng: {question}
            """
        )
    ]
).partial(format_instructions=check_related_parser.get_format_instructions())


# Hàm để xác định xem câu hỏi có liên quan đến giáo dục giới tính không
async def is_sex_education_related(message: str) -> dict:
    llm = ChatOpenAI(
            openai_api_key=OPENAI_API_KEY,
            model="gpt-4o-mini",
            temperature=0.1
        )
    try: 
        chain = check_related_prompt | llm | check_related_parser
        response = chain.invoke({"question" : message})
        return response.model_dump()
    except Exception as e :
        print(f"Error  in check  related : {e}")
        return {"is_sex_education_related" : True, "reason" : str(e)}
        

# Hàm xử lý tin nhắn sử dụng LangChain và GPT-4o-mini
async def process_message_with_langchain(message: str, user_age_group: str, user_gender: str = None, session_id: str = None) -> ChatResponse:
    # Kiểm tra sự phù hợp của nội dung
    if not is_content_appropriate(message, user_age_group):
        return ChatResponse(
            response="Xin lỗi, tôi không thể trả lời câu hỏi này vì nó có thể chứa nội dung không phù hợp.",
            appropriate=False,
            is_sex_education_related=True
        )
    
    try:
        # Kiểm tra API key
        if not OPENAI_API_KEY:
            return ChatResponse(
                response="Thiết lập API chưa hoàn tất. Vui lòng cung cấp OPENAI_API_KEY trong biến môi trường.",
                appropriate=True,
                is_sex_education_related=True
            )
        
        # Xác định xem câu hỏi có liên quan đến giáo dục giới tính không
        sex_ed_result = await is_sex_education_related(message)
        is_related = sex_ed_result["is_sex_education_related"]
        
        # Nếu không liên quan đến giáo dục giới tính
        if not is_related:
            return ChatResponse(
                response=f"Câu hỏi của bạn không liên quan đến giáo dục giới tính. Tôi là một chatbot chuyên về giáo dục giới tính. Lý do: {sex_ed_result['reason']}",
                appropriate=True,
                is_sex_education_related=False
            )
        
        # Khởi tạo LLM với mô hình GPT-4o-mini
        llm = ChatOpenAI(
            openai_api_key=OPENAI_API_KEY,
            model="gpt-4o-mini",
            temperature=0.7
        )
        
        # Tạo hoặc lấy bộ nhớ hội thoại cho người dùng
        if session_id:
            if session_id not in conversation_memories:
                conversation_memories[session_id] = ConversationBufferMemory(return_messages=True)
            memory = conversation_memories[session_id]
        else:
            memory = ConversationBufferMemory(return_messages=True)
        
        # Tạo prompt template dựa trên nhóm tuổi và giới tính người dùng
        if user_gender and user_gender in ["male", "female"]:
            prompt_template = PromptTemplate(
                input_variables=["question"],
                template=age_gender_prompts[user_age_group][user_gender]
            )
        else:
            prompt_template = PromptTemplate(
                input_variables=["question"],
                template=age_gender_prompts[user_age_group]["default"]
            )
        
        # Tạo chain
        chain = LLMChain(
            llm=llm,
            prompt=prompt_template,
            memory=memory
        )
        
        # Gửi tin nhắn và nhận phản hồi
        response = chain.run(question=message)
        
        return ChatResponse(
            response=response,
            appropriate=True,
            is_sex_education_related=True
        )
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return ChatResponse(
            response=f"Đã xảy ra lỗi khi xử lý yêu cầu: {str(e)}",
            appropriate=True,
            is_sex_education_related=True
        )

# API endpoint để xử lý tin nhắn chat
@app.post("/chat", response_model=ChatResponse)
async def chat(request: ChatRequest):
    # Kiểm tra độ tuổi hợp lệ
    if request.user_age_group not in ["child", "teen", "adult", "parent"]:
        raise HTTPException(status_code=400, detail="Độ tuổi không hợp lệ. Vui lòng chọn một trong các giá trị: child, teen, adult, parent")
    
    # Xử lý tin nhắn và trả về câu trả lời sử dụng LangChain và GPT-4o-mini
    return await process_message_with_langchain(request.message, request.user_age_group, request.user_gender, request.session_id)

# API endpoint để lấy danh sách các nhóm tuổi
@app.get("/age-groups", response_model=List[str])
async def get_age_groups():
    return list(age_gender_prompts.keys())

# API endpoint để xóa bộ nhớ hội thoại
@app.delete("/clear-memory/{session_id}")
async def clear_memory(session_id: str):
    if session_id in conversation_memories:
        del conversation_memories[session_id]
        return {"status": "success", "message": f"Đã xóa bộ nhớ hội thoại cho session {session_id}"}
    else:
        raise HTTPException(status_code=404, detail=f"Không tìm thấy session {session_id}")

# API endpoint để kiểm tra trạng thái hoạt động của API
@app.get("/health")
async def health_check():
    api_status = "OK" if OPENAI_API_KEY else "CONFIGURED_ERROR"
    message = "API đang hoạt động bình thường" if OPENAI_API_KEY else "Thiếu OPENAI_API_KEY"
    
    return {
        "status": api_status,
        "message": message,
        "langchain_status": "enabled",
        "model": "gpt-4o-mini"
    }

# API endpoint để lấy thông tin về API
@app.get("/")
async def root():
    return {
        "name": "Sex Education Chatbot API - Powered by GPT-4o-mini",
        "version": "1.0.0",
        "description": "API cung cấp thông tin về giáo dục giới tính phù hợp với độ tuổi sử dụng LangChain và GPT-4o-mini",
        "endpoints": [
            {"path": "/chat", "method": "POST", "description": "Gửi tin nhắn và nhận câu trả lời"},
            {"path": "/age-groups", "method": "GET", "description": "Lấy danh sách các nhóm tuổi"},
            {"path": "/clear-memory/{session_id}", "method": "DELETE", "description": "Xóa bộ nhớ hội thoại cho một session"},
            {"path": "/health", "method": "GET", "description": "Kiểm tra trạng thái hoạt động của API"},
            {"path": "/docs", "method": "GET", "description": "Tài liệu API tự động tạo"}
        ]
    }

# Middleware để xử lý CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Trong môi trường sản xuất, nên chỉ định các nguồn cụ thể
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)



In [2]:

a = await is_sex_education_related("xin chào")
print(a)

{'is_sex_education_related': False, 'reason': 'Câu hỏi không liên quan đến giáo dục giới tính.'}


In [8]:
b = await process_message_with_langchain("What is sex education?", "child", "male","1")
print(b)

response='Giáo dục giới tính là việc học hỏi về cơ thể của chúng ta, cách mà chúng ta phát triển và cách mà chúng ta tôn trọng bản thân cũng như người khác. Đặc biệt, giáo dục giới tính giúp chúng ta hiểu về các thay đổi mà cơ thể con trai sẽ trải qua khi lớn lên, như sự phát triển của cơ bắp, giọng nói thay đổi, và các dấu hiệu khác khi trở thành thanh niên.\n\nNó cũng dạy cho chúng ta về ranh giới cá nhân, nghĩa là biết cách tôn trọng không gian và cơ thể của chính mình và của người khác. Điều quan trọng là luôn cảm thấy thoải mái với cơ thể mình và không ngại hỏi những câu hỏi với người lớn mà bạn tin tưởng.\n\nNếu bạn có thêm câu hỏi nào khác, hãy cứ hỏi nhé!' appropriate=True is_sex_education_related=True


In [9]:
print(conversation_memories)

{'1': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Giáo dục giới tính là gì', additional_kwargs={}, response_metadata={}), AIMessage(content='Giáo dục giới tính là việc học về cơ thể của chúng ta, cách chúng ta phát triển và cảm xúc của mình. Nó giúp chúng ta hiểu về bản thân, biết tôn trọng cơ thể của mình và cơ thể của người khác. \n\nTrong giáo dục giới tính, chúng ta có thể học về những thay đổi mà cơ thể nam giới trải qua khi lớn lên, như là sự phát triển của cơ bắp, sự thay đổi giọng nói, và sự xuất hiện của lông ở một số nơi trên cơ thể. \n\nĐiều quan trọng là chúng ta cũng học về ranh giới cá nhân, nghĩa là biết cách tôn trọng không gian và cảm xúc của chính mình và người khác. Nếu có điều gì thắc mắc, hãy hỏi người lớn mà bạn tin tưởng nhé!', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is sex education?', additional_kwargs={}, response_metadata={}), AIMessage(content='Giáo dục giới tính là một cách 